# Homework 5

Averaging multiple copies of a signal in the presence of different realizations of noise can improve the SNR, and thus accuracy of detecting a signal.  In real world situations, it is often impossible to get identical copies of the signal, but it is quite possible to get similar copies where the difference between observations is known.  One of the most common ways to achieve this is to use a line array of sensors.  If the signal is assumed to be propagating as a plane wave, the time of arrival at each sensor will differ only by a constant.  Accounting for these time delays, it is then possible to combine identical signals with different noise realizations, yielding a higher SNR.

## 1. 
$$ f_0(\pmb{y}) = \frac{1}{\sqrt{det(2\sigma_{Z_i}^2)}}e^{-\pmb{y}'\pmb{y}}$$
$$ f_1(\pmb{y}) = \frac{1}{\sqrt{det(2\sigma_{Z_i}^2)}}e^{-(\pmb{y}-A\pmb{s})'(\pmb{y}-A\pmb{s})}$$

## 2.
$$\frac{f_1(\pmb{y})}{f_0(\pmb{y})} = \frac{e^{-(\pmb{y}-A\pmb{s})'(\pmb{y}-A\pmb{s})}}{e^{-\pmb{y}'\pmb{y}}} \gtrless \eta$$

## 3.
$$ln(\eta) \gtrless -(\pmb{y}-A\pmb{s})'(\pmb{y}-A\pmb{s})+y'y$$
$$ln(\eta) \gtrless -\pmb{y}'\pmb{y}+\pmb{y}'A\pmb{s}+A\pmb{s}'\pmb{y}-A^2\pmb{s}'\pmb{s}$$
$$A[\pmb{y}'\pmb{s} + \pmb{s}'\pmb{y}] \gtrless A^2\pmb{s}'\pmb{s} + ln(\eta)$$

$\pmb{y's}$ and $\pmb{s'y}$ are complex conjugates if $s$ and $y$ are complex, so $\pmb{y's+s'y = 2Re(y's)}$  
$\pmb{(y's)' = s'y}$ and $\pmb{y's}$ is a scalar, so it is valid to write

$$Re(A\pmb{s}'\pmb{y}) \gtrless \frac{1}{2}A\pmb{s}'C_z^{-1}A\pmb{s} + ln(\eta)$$

It is convenient to remove the real constant $A$ from the left side of the equation and divide by the number of sensors so that the combined measurement is on the same scale as an individual observation.  Doing this yields

$$\frac{1}{M} Re(\pmb{s'y}) \gtrless \gamma$$

## 4.
$\frac{1}{M}Re(\pmb{s'y})$ is a normal distribution described by 
$$
\frac{1}{M}Re(\pmb{s'y}) =
\begin{cases}
\mathcal{CN}\big(0, \frac{\sigma_{Z_i}}{2M}\big) & \pmb{y=z} \\
\mathcal{CN}\big(A, \frac{\sigma_{Z_i}}{2M}\big) & \pmb{y}=A+\pmb{z}
\end{cases}
$$
To find $\gamma$, we must calculate the area under the right tail of this distribution when $\pmb{y=z}$.
$$\gamma = \sqrt{\frac{\sigma_{Z_i}^2}{2M}}Q(P_{FA}^*)$$

## 5.

In [12]:
pkg load statistics
pkg load communications

In [13]:
load hwk5.mat

In [14]:
function retval = lin_arr_det(vecs, angle, distance, wavelength, noise_var,  pfamax)
    M = size(vecs)(1);
    phi = 2*pi*distance/wavelength * sind(angle);
    s = exp(-i .* (0:M-1)' .* phi);
    newvar = sqrt(noise_var / (2*M));
    thresh = newvar * qfuncinv(pfamax);
    retval = real(s'*vecs)/M > thresh;
end

In [15]:
res = find(lin_arr_det(thousand_vectors_y, 20, 1, 2, 1, 0.1));
size(res)(2)

ans =  107


The above code finds 107 detections.  The $10\%$ false alarm rate specified should give about 100 detections, so assuming the signal is rarely present 107 detections sounds plausible.

## 6.

In [23]:
truth = [148 607 624 626 645 711 727 780];
function [cd, md] = inres(x, res)
    cd = 0;
    for k=x
        if sum(ismember(res, k)) ==1
            fprintf("%d Correctly Detected\n", k);
            cd += 1;
        else
            fprintf("%d Missed Detection\n", k);
        end
    end
    md = size(x)(2) - cd;
end

[cd, md] = inres(truth, res)

148 Correctly Detected
607 Missed Detection
624 Correctly Detected
626 Correctly Detected
645 Correctly Detected
711 Missed Detection
727 Correctly Detected
780 Correctly Detected
cd =  6
md =  2


The model correctly classified 6 out of the 8 signals.  Missed detections occurred at samples 607 and 711.